In [ ]:
"""
# Mounting Drive
from google.colab import drive
drive.mount('/content/drive')


#---------------------------------------------------------
# Necessary imports
import glob
import pandas as pd
import urllib.request
import imageio
import os
import numpy as np

from urllib.request import urlopen


#---------------------------------------------------------
# Load Glove Function
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r',encoding="utf8")
    model = {}
    for line in f:
        try:
            splitLine = line.split()
            word = splitLine[0]
            embedding = np.array([float(val) for val in splitLine[1:]])
            model[word] = embedding
        except:
            print(word)
    print("Done.",len(model)," words loaded!")
    return model

#---------------------------------------------------------    

glove_embeddings = loadGloveModel("/content/drive/MyDrive/glove.6B.300d.txt")

#---------------------------------------------------------    
# Setting up the paths
train_data_path = "/content/drive/MyDrive/flowers"
train_images_path = "/content/drive/MyDrive/flowers/jpg"
train_captions_path = "/content/drive/MyDrive/flowers/text_c10"

#---------------------------------------------------------    
# Imports
import tensorflow as tf
from tensorflow.keras.layers import Input, Reshape, Dropout, Dense, Concatenate 
from tensorflow.keras.layers import Flatten, BatchNormalization
from tensorflow.keras.layers import Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D, Conv2DTranspose
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras import initializers
from sklearn.metrics import mean_squared_error

import numpy as np
from PIL import Image
from tqdm import tqdm
import os 
import time
import matplotlib.pyplot as plt
#--------------------------------------------------------- 


# Formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)
     
#---------------------------------------------------------    
# Generation resolution - Must be square 
# Training data is also scaled to this.
GENERATE_RES = 2 # Generation resolution factor 
# (1=32, 2=64, 3=96, 4=128, etc.)
GENERATE_SQUARE = 32 * GENERATE_RES # rows/cols (should be square)
IMAGE_CHANNELS = 3

# Preview image 
PREVIEW_ROWS = 4
PREVIEW_COLS = 7
PREVIEW_MARGIN = 16

# Size vector to generate images from
SEED_SIZE = 100
EMBEDDING_SIZE = 300

# Configuration
DATA_PATH = train_images_path
MODEL_PATH = "/content/drive/Shareddrives/D4NLP Project/flowers data/flowers/model"
EPOCHS = 50
BATCH_SIZE = 64
BUFFER_SIZE = 4000

print(f"Will generate {GENERATE_SQUARE}px square images.")

#---------------------------------------------------------    

# Image set has 8,188 images.  Can take over an hour 
# for initial preprocessing.
# Because of this time needed, save a Numpy preprocessed file.

training_binary_path = os.path.join("/content/drive/Shareddrives/D4NLP Project/flowers data/flowers/images/npy64",
        f'training_data_{GENERATE_SQUARE}_{GENERATE_SQUARE}_')

start = time.time()
print("Loading training images...")

training_data = []
flowers_path = sorted(os.listdir(DATA_PATH))

for filename in range(len(flowers_path)):
    path = os.path.join(DATA_PATH,flowers_path[filename])
    # print(path)
    try:
      image = Image.open(path).resize((GENERATE_SQUARE,
            GENERATE_SQUARE),Image.ANTIALIAS)
      channel = np.asarray(image).shape[2]
      if channel == 3:
        training_data.append(np.asarray(image))
    except KeyboardInterrupt:
        print("Keyboard Interrup by me...")
        break
    except:
      pass
    if len(training_data) == 100:
      training_data = np.reshape(training_data,(-1,GENERATE_SQUARE,
                GENERATE_SQUARE,IMAGE_CHANNELS))
      training_data = training_data.astype(np.float32)
      #Normalizing the input
      training_data = training_data / 127.5 - 1.

      print("Saving training image " + str(100000 + filename) + ".npy")
      np.save(training_binary_path + str(100000 + filename) + ".npy",training_data)
      elapsed = time.time()-start
      print (f'Image preprocess time: {hms_string(elapsed)}')
      training_data = []
print("Complete")

"""

'\n# Mounting Drive\nfrom google.colab import drive\ndrive.mount(\'/content/drive\')\n\n\n#---------------------------------------------------------\n# Necessary imports\nimport glob\nimport pandas as pd\nimport urllib.request\nimport imageio\nimport os\nimport numpy as np\n\nfrom urllib.request import urlopen\n\n\n#---------------------------------------------------------\n# Load Glove Function\ndef loadGloveModel(gloveFile):\n    print("Loading Glove Model")\n    f = open(gloveFile,\'r\',encoding="utf8")\n    model = {}\n    for line in f:\n        try:\n            splitLine = line.split()\n            word = splitLine[0]\n            embedding = np.array([float(val) for val in splitLine[1:]])\n            model[word] = embedding\n        except:\n            print(word)\n    print("Done.",len(model)," words loaded!")\n    return model\n\n#---------------------------------------------------------    \n\nglove_embeddings = loadGloveModel("/content/drive/MyDrive/glove.6B.300d.txt")\n\

In [ ]:
!pip install -U featuretools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install opendatasets --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import files
files.upload()

{}

In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!ls

bi_skip.npz	 dictionary.txt		   images1.pickle  sample_data
bi_skip.npz.pkl  drive			   images2.pickle  uni_skip.npz
btable.npy	 flickr30k_images	   images3.pickle  uni_skip.npz.pkl
captions.pickle  flickr-image-dataset.zip  kaggle.json	   utable.npy


In [ ]:
!kaggle datasets download -d hsankesara/flickr-image-dataset

flickr-image-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!ls

bi_skip.npz	 dictionary.txt		   images1.pickle  sample_data
bi_skip.npz.pkl  drive			   images2.pickle  uni_skip.npz
btable.npy	 flickr30k_images	   images3.pickle  uni_skip.npz.pkl
captions.pickle  flickr-image-dataset.zip  kaggle.json	   utable.npy


In [ ]:
!unzip flickr-image-dataset.zip

Archive:  flickr-image-dataset.zip
replace flickr30k_images/flickr30k_images/1000092795.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!ls

bi_skip.npz	 dictionary.txt		   images1.pickle  sample_data
bi_skip.npz.pkl  drive			   images2.pickle  uni_skip.npz
btable.npy	 flickr30k_images	   images3.pickle  uni_skip.npz.pkl
captions.pickle  flickr-image-dataset.zip  kaggle.json	   utable.npy


In [ ]:
# cd flickr30k_images/flickr30k_images

In [ ]:
# cd -

In [ ]:
pwd # /content/flickr30k_images/flickr30k_images

'/content'

In [ ]:
!ls

 drive		    flickr-image-dataset.zip   kaggle.json
 flickr30k_images  'kaggle (1).json'	       sample_data


In [ ]:
!pip install theano

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Caption.py

In [ ]:
from zipfile import ZipFile
from pandas import read_csv
from io import BytesIO
import numpy as np
from pickle import dump, load

from PIL import Image
from cv2 import resize

import os
import torch
import theano
import theano.tensor as tensor

import pickle as pkl
import numpy
import copy
import nltk

from collections import OrderedDict, defaultdict
from scipy.linalg import norm
from nltk.tokenize import word_tokenize

from torch.autograd import Variable
from torch.utils.data import Dataset

import torch.nn as nn
from torch.nn import functional as F

import time
import datetime
import logging
import itertools

import torch.optim as optim
from torchvision.utils import save_image

In [ ]:
archive = ZipFile(file="/content/flickr-image-dataset.zip", mode="r")
# archive.printdir() # prints all the contents of the zip file

In [ ]:
df = read_csv("/content/flickr30k_images/flickr30k_images/results.csv", sep="|")
df = df.rename(columns=lambda x: x.strip())

In [ ]:
df

,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their...
1,1000092795.jpg,1,"Two young , White males are outside near many..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard .
3,1000092795.jpg,3,A man in a blue shirt standing in a garden .
4,1000092795.jpg,4,Two friends enjoy time spent together .
...,...,...,...
158910,998845445.jpg,0,A man in shorts and a Hawaiian shirt leans ov...
158911,998845445.jpg,1,"A young man hanging over the side of a boat ,..."
158912,998845445.jpg,2,A man is leaning off of the side of a blue an...
158913,998845445.jpg,3,"A man riding a small boat in a harbor , with ..."


In [ ]:
IMAGELIST = df.loc[:, "image_name"].unique()
captions = {}
print(IMAGELIST)
print(len(IMAGELIST))

['1000092795.jpg' '10002456.jpg' '1000268201.jpg' ... '997876722.jpg'
 '99804383.jpg' '998845445.jpg']
31783


In [ ]:
for (idx, label) in enumerate(IMAGELIST):
  #print(idx, label)
  captions[idx] = list(df.loc[df["image_name"] == label, "comment"].values)

In [ ]:
captions[0]

[' Two young guys with shaggy hair look at their hands while hanging out in the yard .',
 ' Two young , White males are outside near many bushes .',
 ' Two men in green shirts are standing in a yard .',
 ' A man in a blue shirt standing in a garden .',
 ' Two friends enjoy time spent together .']

In [ ]:
captions

{}

In [ ]:
# captions.keys()

In [ ]:
# Dictionary storing all the captions related to particular images
with open('captions.pickle', 'wb') as file:
    dump(captions, file)

del file

# storing all the captions according to keys()
result = [captions[key] for key in sorted(captions.keys())]
with open('captions.pickle', 'wb') as file:
    dump(result, file)

del file

In [ ]:
result[0]

[' Two young guys with shaggy hair look at their hands while hanging out in the yard .',
 ' Two young , White males are outside near many bushes .',
 ' Two men in green shirts are standing in a yard .',
 ' A man in a blue shirt standing in a garden .',
 ' Two friends enjoy time spent together .']

# Image.py

In [ ]:
# DIRECTORY : /content/flickr30k_images/flickr30k_images
print(IMAGELIST)
archive = ZipFile(file="/content/flickr-image-dataset.zip", mode="r")

['1000092795.jpg' '10002456.jpg' '1000268201.jpg' ... '997876722.jpg'
 '99804383.jpg' '998845445.jpg']


# ---------------------------------------------------------------------------------------------------------------------
### Dividing the whole dataset into 3 parts - images1.pickle, images2.pickle, images3.pickle

#### 0-10000 - images1.pickle
#### 10000-20000 - images2.pickle
#### 20000-Remaining - images3.pickle

In [ ]:

images = {}
start_idx = 0
end_idx = 10000


for (idx, label) in enumerate(IMAGELIST[start_idx : end_idx]):

    img = archive.read("flickr30k_images/flickr30k_images/{}".format(label))
    img = BytesIO(img)
    img = np.asarray(Image.open(img))
    img = resize(img, (300,300))

    images[idx+start_idx] = img

result = np.array([images[key] for key in sorted(images.keys())])
with open('images1.pickle', 'wb') as file:
    dump(result, file)

del images
del file


In [ ]:
images = {}
start_idx = 10000
end_idx = 20000


for (idx, label) in enumerate(IMAGELIST[start_idx : end_idx]):

    img = archive.read("flickr30k_images/flickr30k_images/{}".format(label))
    img = BytesIO(img)
    img = np.asarray(Image.open(img))
    img = resize(img, (300,300))

    images[idx+start_idx] = img

result = np.array([images[key] for key in sorted(images.keys())])
with open('images2.pickle', 'wb') as file:
    dump(result, file)

del images
del file

In [ ]:
images = {}
start_idx = 20000
end_idx = None


for (idx, label) in enumerate(IMAGELIST[start_idx : ]):

    img = archive.read("flickr30k_images/flickr30k_images/{}".format(label))
    img = BytesIO(img)
    img = np.asarray(Image.open(img))
    img = resize(img, (300,300))

    images[idx+start_idx] = img

result = np.array([images[key] for key in sorted(images.keys())])
with open('images3.pickle', 'wb') as file:
    dump(result, file)




del file
del images
del result

# Till this point we had done a serialization of the Flickr Dataset

Serialization refers to the process of converting a data object (e.g., Python objects, Tensorflow models) into a format that allows us to store or transmit the data and then recreate the object when needed using the reverse process of deserialization

# Skip_thoughts.py

https://github.com/itsayushthada/Text-To-Image

https://github.com/ryankiros/skip-thoughts

https://arxiv.org/pdf/1605.05396.pdf

In [ ]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=ec8f4156e674c41790441f233b3b8c18702bd3889629f2ff79bd98284bb7d84d
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget


In [ ]:
import wget
wget.download('http://www.cs.toronto.edu/~rkiros/models/dictionary.txt')
wget.download('http://www.cs.toronto.edu/~rkiros/models/utable.npy')
wget.download('http://www.cs.toronto.edu/~rkiros/models/btable.npy')
wget.download('http://www.cs.toronto.edu/~rkiros/models/uni_skip.npz')
wget.download('http://www.cs.toronto.edu/~rkiros/models/bi_skip.npz')
wget.download('http://www.cs.toronto.edu/~rkiros/models/uni_skip.npz.pkl')
wget.download('http://www.cs.toronto.edu/~rkiros/models/bi_skip.npz.pkl')

'bi_skip.npz.pkl'

In [ ]:

profile = False

#-----------------------------------------------------------------------------#
# Specify model and table locations here
#-----------------------------------------------------------------------------#
path_to_models = '/content/'
path_to_tables = '/content'
#-----------------------------------------------------------------------------#

path_to_umodel = path_to_models + 'uni_skip.npz'
path_to_bmodel = path_to_models + 'bi_skip.npz'


def load_model():
    """
    Load the model with saved tables
    """
    # Load model options
    print('Loading model parameters...')
    with open('%s.pkl' % path_to_umodel, 'rb') as f:
        uoptions = pkl.load(f)
    with open('%s.pkl' % path_to_bmodel, 'rb') as f:
        boptions = pkl.load(f)

    # Load parameters
    uparams = init_params(uoptions)
    uparams = load_params(path_to_umodel, uparams)
    utparams = init_tparams(uparams)
    bparams = init_params_bi(boptions)
    bparams = load_params(path_to_bmodel, bparams)
    btparams = init_tparams(bparams)

    # Extractor functions
    print('Compiling encoders...')
    embedding, x_mask, ctxw2v = build_encoder(utparams, uoptions)
    f_w2v = theano.function([embedding, x_mask], ctxw2v, name='f_w2v')
    embedding, x_mask, ctxw2v = build_encoder_bi(btparams, boptions)
    f_w2v2 = theano.function([embedding, x_mask], ctxw2v, name='f_w2v2')

    # Tables
    print('Loading tables...')
    utable, btable = load_tables()

    # Store everything we need in a dictionary
    print('Packing up...')
    model = {}
    model['uoptions'] = uoptions
    model['boptions'] = boptions
    model['utable'] = utable
    model['btable'] = btable
    model['f_w2v'] = f_w2v
    model['f_w2v2'] = f_w2v2

    return model

	
	

def load_tables():
    """
    Load the tables
    """
    words = []
    utable = numpy.load(path_to_tables + 'utable.npy', encoding='latin1')
    btable = numpy.load(path_to_tables + 'btable.npy', encoding='latin1')
    f = open(path_to_tables + 'dictionary.txt', 'rb')
    for line in f:
        words.append(line.decode('utf-8').strip())
    f.close()
    utable = OrderedDict(list(zip(words, utable)))
    btable = OrderedDict(list(zip(words, btable)))
    return utable, btable

	
	
class Encoder(object):
    """
    Sentence encoder.
    """

    def __init__(self, model):
      self._model = model

    def encode(self, X, use_norm=True, verbose=True, batch_size=128, use_eos=False):
      """
      Encode sentences in the list X. Each entry will return a vector
      """
      return encode(self._model, X, use_norm, verbose, batch_size, use_eos)	

	  
	  
def encode(model, X, use_norm=True, verbose=False, batch_size=128, use_eos=False):
    """
    Encode sentences in the list X. Each entry will return a vector
    """
    # first, do preprocessing
    X = preprocess(X)

    # word dictionary and init
    d = defaultdict(lambda: 0)
    for w in list(model['utable'].keys()):
        d[w] = 1
    ufeatures = numpy.zeros((len(X), model['uoptions']['dim']), dtype='float32')
    bfeatures = numpy.zeros((len(X), 2 * model['boptions']['dim']), dtype='float32')

    # length dictionary
    ds = defaultdict(list)
    captions = [s.split() for s in X]
    for i, s in enumerate(captions):
        ds[len(s)].append(i)

    # Get features. This encodes by length, in order to avoid wasting computation
    for k in list(ds.keys()):
        if verbose:
            print(k)
        numbatches = len(ds[k]) // batch_size + 1
        for minibatch in range(numbatches):
            caps = ds[k][minibatch::numbatches]

            if use_eos:
                uembedding = numpy.zeros((k + 1, len(caps), model['uoptions']['dim_word']), dtype='float32')
                bembedding = numpy.zeros((k + 1, len(caps), model['boptions']['dim_word']), dtype='float32')
            else:
                uembedding = numpy.zeros((k, len(caps), model['uoptions']['dim_word']), dtype='float32')
                bembedding = numpy.zeros((k, len(caps), model['boptions']['dim_word']), dtype='float32')
            for ind, c in enumerate(caps):
                caption = captions[c]
                for j in range(len(caption)):
                    if d[caption[j]] > 0:
                        uembedding[j, ind] = model['utable'][caption[j]]
                        bembedding[j, ind] = model['btable'][caption[j]]
                    else:
                        uembedding[j, ind] = model['utable']['UNK']
                        bembedding[j, ind] = model['btable']['UNK']
                if use_eos:
                    uembedding[-1, ind] = model['utable']['<eos>']
                    bembedding[-1, ind] = model['btable']['<eos>']
            if use_eos:
                uff = model['f_w2v'](uembedding, numpy.ones((len(caption) + 1, len(caps)), dtype='float32'))
                bff = model['f_w2v2'](bembedding, numpy.ones((len(caption) + 1, len(caps)), dtype='float32'))
            else:
                uff = model['f_w2v'](uembedding, numpy.ones((len(caption), len(caps)), dtype='float32'))
                bff = model['f_w2v2'](bembedding, numpy.ones((len(caption), len(caps)), dtype='float32'))
            if use_norm:
                for j in range(len(uff)):
                    uff[j] /= norm(uff[j])
                    bff[j] /= norm(bff[j])
            for ind, c in enumerate(caps):
                ufeatures[c] = uff[ind]
                bfeatures[c] = bff[ind]

    features = numpy.c_[ufeatures, bfeatures]
    return features

	

def preprocess(text):
    """
    Preprocess text for encoder
    """
    X = []
    sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
    for t in text:
        sents = sent_detector.tokenize(t)
        result = ''
        for s in sents:
            tokens = word_tokenize(s)
            result += ' ' + ' '.join(tokens)
        X.append(result)
    return X


	
def nn(model, text, vectors, query, k=5):
    """
    Return the nearest neighbour sentences to query
    text: list of sentences
    vectors: the corresponding representations for text
    query: a string to search
    """
    qf = encode(model, [query])
    qf /= norm(qf)
    scores = numpy.dot(qf, vectors.T).flatten()
    sorted_args = numpy.argsort(scores)[::-1]
    sentences = [text[a] for a in sorted_args[:k]]
    print(('QUERY: ' + query))
    print('NEAREST: ')
    for i, s in enumerate(sentences):
        print((s, sorted_args[i]))


		
def word_features(table):
    """
    Extract word features into a normalized matrix
    """
    features = numpy.zeros((len(table), 620), dtype='float32')
    keys = list(table.keys())
    for i in range(len(table)):
        f = table[keys[i]]
        features[i] = f / norm(f)
    return features

	

def nn_words(table, wordvecs, query, k=10):
    """
    Get the nearest neighbour words
    """
    keys = list(table.keys())
    qf = table[query]
    scores = numpy.dot(qf, wordvecs.T).flatten()
    sorted_args = numpy.argsort(scores)[::-1]
    words = [keys[a] for a in sorted_args[:k]]
    print(('QUERY: ' + query))
    print('NEAREST: ')
    for i, w in enumerate(words):
        print(w)


		
def _p(pp, name):
    """
    make prefix-appended name
    """
    return '%s_%s' % (pp, name)


	
def init_tparams(params):
    """
    initialize Theano shared variables according to the initial parameters
    """
    tparams = OrderedDict()
    for kk, pp in list(params.items()):
        tparams[kk] = theano.shared(params[kk], name=kk)
    return tparams


	
def load_params(path, params):
    """
    load parameters
    """
    pp = numpy.load(path)
    for kk, vv in list(params.items()):
        if kk not in pp:
            warnings.warn('%s is not in the archive' % kk)
            continue
        params[kk] = pp[kk]
    return params


# layers: 'name': ('parameter initializer', 'feedforward')
layers = {'gru': ('param_init_gru', 'gru_layer')}



def get_layer(name):
    fns = layers[name]
    return (eval(fns[0]), eval(fns[1]))


	
def init_params(options):
    """
    initialize all parameters needed for the encoder
    """
    params = OrderedDict()

    # embedding
    params['Wemb'] = norm_weight(options['n_words_src'], options['dim_word'])

    # encoder: GRU
    params = get_layer(options['encoder'])[0](options, params, prefix='encoder',
                                              nin=options['dim_word'], dim=options['dim'])
    return params

	

def init_params_bi(options):
    """
    initialize all paramters needed for bidirectional encoder
    """
    params = OrderedDict()

    # embedding
    params['Wemb'] = norm_weight(options['n_words_src'], options['dim_word'])

    # encoder: GRU
    params = get_layer(options['encoder'])[0](options, params, prefix='encoder',
                                              nin=options['dim_word'], dim=options['dim'])
    params = get_layer(options['encoder'])[0](options, params, prefix='encoder_r',
                                              nin=options['dim_word'], dim=options['dim'])
    return params


	
def build_encoder(tparams, options):
    """
    build an encoder, given pre-computed word embeddings
    """
    # word embedding (source)
    embedding = tensor.tensor3('embedding', dtype='float32')
    x_mask = tensor.matrix('x_mask', dtype='float32')

    # encoder
    proj = get_layer(options['encoder'])[1](tparams, embedding, options,
                                            prefix='encoder',
                                            mask=x_mask)
    print("------------------------->")
    print("------------------------->")
    print(proj)
    ctx = proj[0][-1]

    return embedding, x_mask, ctx


	
def build_encoder_bi(tparams, options):
    """
    build bidirectional encoder, given pre-computed word embeddings
    """
    # word embedding (source)
    embedding = tensor.tensor3('embedding', dtype='float32')
    embeddingr = embedding[::-1]
    x_mask = tensor.matrix('x_mask', dtype='float32')
    xr_mask = x_mask[::-1]

    # encoder
    proj = get_layer(options['encoder'])[1](tparams, embedding, options,
                                            prefix='encoder',
                                            mask=x_mask)
    projr = get_layer(options['encoder'])[1](tparams, embeddingr, options,
                                             prefix='encoder_r',
                                             mask=xr_mask)

    ctx = tensor.concatenate([proj[0][-1], projr[0][-1]], axis=1)

    return embedding, x_mask, ctx


# some utilities
def ortho_weight(ndim):
    W = numpy.random.randn(ndim, ndim)
    u, s, v = numpy.linalg.svd(W)
    return u.astype('float32')


	
def norm_weight(nin, nout=None, scale=0.1, ortho=True):
    if nout == None:
        nout = nin
    if nout == nin and ortho:
        W = ortho_weight(nin)
    else:
        W = numpy.random.uniform(low=-scale, high=scale, size=(nin, nout))
    return W.astype('float32')


	
def param_init_gru(options, params, prefix='gru', nin=None, dim=None):
    """
    parameter init for GRU
    """
    if nin == None:
        nin = options['dim_proj']
    if dim == None:
        dim = options['dim_proj']
    W = numpy.concatenate([norm_weight(nin, dim),
                           norm_weight(nin, dim)], axis=1)
    params[_p(prefix, 'W')] = W
    params[_p(prefix, 'b')] = numpy.zeros((2 * dim,)).astype('float32')
    U = numpy.concatenate([ortho_weight(dim),
                           ortho_weight(dim)], axis=1)
    params[_p(prefix, 'U')] = U

    Wx = norm_weight(nin, dim)
    params[_p(prefix, 'Wx')] = Wx
    Ux = ortho_weight(dim)
    params[_p(prefix, 'Ux')] = Ux
    params[_p(prefix, 'bx')] = numpy.zeros((dim,)).astype('float32')

    return params


	
def gru_layer(tparams, state_below, options, prefix='gru', mask=None, **kwargs):
    """
    Forward pass through GRU layer
    """
    nsteps = state_below.shape[0]
    if state_below.ndim == 3:
        n_samples = state_below.shape[1]
    else:
        n_samples = 1

    dim = tparams[_p(prefix, 'Ux')].shape[1]

    if mask == None:
        mask = tensor.alloc(1., state_below.shape[0], 1)

    def _slice(_x, n, dim):
        if _x.ndim == 3:
            return _x[:, :, n * dim:(n + 1) * dim]
        return _x[:, n * dim:(n + 1) * dim]

    state_below_ = tensor.dot(state_below, tparams[_p(prefix, 'W')]) + tparams[_p(prefix, 'b')]
    state_belowx = tensor.dot(state_below, tparams[_p(prefix, 'Wx')]) + tparams[_p(prefix, 'bx')]
    U = tparams[_p(prefix, 'U')]
    Ux = tparams[_p(prefix, 'Ux')]

    def _step_slice(m_, x_, xx_, h_, U, Ux):
        preact = tensor.dot(h_, U)
        preact += x_

        r = tensor.nnet.sigmoid(_slice(preact, 0, dim))
        u = tensor.nnet.sigmoid(_slice(preact, 1, dim))

        preactx = tensor.dot(h_, Ux)
        preactx = preactx * r
        preactx = preactx + xx_

        h = tensor.tanh(preactx)

        h = u * h_ + (1. - u) * h
        h = m_[:, None] * h + (1. - m_)[:, None] * h_

        return h

    seqs = [mask, state_below_, state_belowx]
    _step = _step_slice

    rval, updates = theano.scan(_step,
                                sequences=seqs,
                                outputs_info=[tensor.alloc(0., n_samples, dim)],
                                non_sequences=[tparams[_p(prefix, 'U')],
                                               tparams[_p(prefix, 'Ux')]],
                                name=_p(prefix, '_layers'),
                                n_steps=nsteps,
                                profile=profile,
                                strict=True)
    rval = [rval]


    return 

# Data_Loader.py

https://github.com/prakashpandey9/Text2Image-PyTorch/blob/master/data_loader.py

In [ ]:
class Text2ImageDataset(Dataset):

	def __init__(self, img_dir = "/content/images1.pickle", cap_dir = "/content/captions.pickle"):
		self.images = self.load_pickle(img_dir)
		self.captions = self.load_pickle(cap_dir)
		# self.model = load_model()

	def load_pickle(self, pickle_path):
		obj = 0
		root_path = ""
		with open(root_path + pickle_path, "rb") as inputfile:
			obj = load(inputfile)  
		return obj

	def encode_captions(self, captions_list):
		fun = lambda x: encode(self.model, x) 
		encoded_list = list(map(fun, captions_list))
		return encoded_list

	def read_image(self, img_idx):
		return self.images[img_idx]

	def false_image(self, img_idx):
		idx = np.random.randint(0, self.__len__())
		if (idx != img_idx):
			return self.images[idx]
		return self.false_image(img_idx)

	def __len__(self):
		return self.images.shape[0]

	def __getitem__(self, idx):
		sample = {}
		sample["true_imgs"] = torch.FloatTensor(self.read_image(idx))
		sample["false_imgs"] = torch.FloatTensor(self.false_image(idx))
		# sample["true_embds"] = torch.FloatTensor(self.encode_captions([self.captions[idx]])[0][np.random.randint(0,5)])
		return sample

In [ ]:
# a = Text2ImageDataset()

# Generator.py

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F

class Generator(nn.Module):
	def __init__(self, batch_size, img_size, z_dim, text_embed_dim, text_reduced_dim):
		super(Generator, self).__init__()

		self.img_size = img_size
		self.z_dim = z_dim
		self.text_embed_dim = text_embed_dim

		self.concat = nn.Linear(z_dim + text_reduced_dim, 64 * 8 * 4 * 4).cuda()
		self.text_reduced_dim = nn.Linear(text_embed_dim, text_reduced_dim).cuda()
		
		# Defining the generator network architecture
		self.d_net = nn.Sequential(
			nn.ReLU(),
			nn.ConvTranspose2d(512, 256, 4, 2, 1),
			nn.BatchNorm2d(256),
			nn.ReLU(),
			nn.ConvTranspose2d(256, 128, 4, 2, 1),
			nn.BatchNorm2d(128),
			nn.ReLU(),
			nn.ConvTranspose2d(128, 64, 4, 2, 1),
			nn.BatchNorm2d(64),
			nn.ReLU(),
			nn.ConvTranspose2d(64, 3, 4, 2, 1),
			nn.Tanh()
		).cuda()

	def forward(self, text, z):
		""" Given a caption embedding and latent variable z(noise), generate an image
		Arguments
		---------
		text : torch.FloatTensor
			Output of the skipthought embedding model for the caption
			text.size() = (batch_size, text_embed_dim)
		z : torch.FloatTensor
			Latent variable or noise
			z.size() = (batch_size, z_dim)
		--------
		Returns
		--------
		output : An image of shape (64, 64, 3)
		"""
		reduced_text = self.text_reduced_dim(text.cuda())  # (batch_size, text_reduced_dim)
		concat = torch.cat((reduced_text, z.cuda()), 1)  # (batch_size, text_reduced_dim + z_dim)
		concat = self.concat(concat)  # (batch_size, 64*8*4*4)
		concat = concat.view(-1, 4, 4, 64 * 8)  # (batch_size, 4, 4, 64*8)
		
		concat = concat.permute(0, 3, 1, 2) # (batch_size, 512, 4, 4)
		d_net_out = self.d_net(concat)  # (batch_size, 3, 64, 64)
		d_net_out = d_net_out.permute(0, 2, 3, 1) #(batch_size, 64, 64, 3)
		
		output = d_net_out / 2. + 0.5   # (batch_size, 64, 64, 3)

		return output

# Discriminator.py

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F

class Discriminator(nn.Module):
	def __init__(self, batch_size, img_size, text_embed_dim, text_reduced_dim):
		super(Discriminator, self).__init__()

		self.batch_size = batch_size
		self.img_size = img_size
		self.in_channels = img_size[2]
		self.text_embed_dim = text_embed_dim
		self.text_reduced_dim = text_reduced_dim

		# Defining the discriminator network architecture
		self.d_net = nn.Sequential(
			nn.Conv2d(self.in_channels, 64, 4, 2, 1, bias=False),
			nn.LeakyReLU(0.2, inplace=True),
			nn.Conv2d(64, 128, 4, 2, 1, bias=False),
			nn.BatchNorm2d(128),
			nn.LeakyReLU(0.2, inplace=True),
			nn.Conv2d(128, 256, 4, 2, 1, bias=False),
			nn.BatchNorm2d(256),
			nn.LeakyReLU(0.2, inplace=True),
			nn.Conv2d(256, 512, 4, 2, 1, bias=False),
			nn.BatchNorm2d(512),
			nn.LeakyReLU(0.2, inplace=True)).cuda()

		# output_dim = (batch_size, 4, 4, 512)
		# text.size() = (batch_size, text_embed_dim)

		# Defining a linear layer to reduce the dimensionality of caption embedding
		# from text_embed_dim to text_reduced_dim

		self.cat_net = nn.Sequential(
			nn.Conv2d(512 + self.text_reduced_dim, 512, 4, 2, 1, bias=False),
			nn.BatchNorm2d(512),
			nn.LeakyReLU(0.2, inplace=True)).cuda()

		self.text_reduced_dim = nn.Linear(self.text_embed_dim, self.text_reduced_dim).cuda()
		
		self.linear = nn.Linear(2 * 2 * 512, 1).cuda()

	def forward(self, image, text):
		""" Given the image and its caption embedding, predict whether the image
		is real or fake.
		Arguments
		---------
		image : torch.FloatTensor
			image.size() = (batch_size, 64, 64, 3)
		text : torch.FloatTensor
			Output of the skipthought embedding model for the caption
			text.size() = (batch_size, text_embed_dim)
		--------
		Returns
		--------
		output : Probability for the image being real/fake
		logit : Final score of the discriminator
		"""
		image = image.permute(0, 3, 1, 2) # (batch_size, 3, 64, 64)
		d_net_out = self.d_net(image)  # (batch_size, 512, 4, 4)
		d_net_out = d_net_out.permute(0, 2, 3, 1) # (batch_size, 4, 4, 512)
		
		text_reduced = self.text_reduced_dim(text)  # (batch_size, text_reduced_dim)
		text_reduced = text_reduced.unsqueeze(1)  # (batch_size, 1, text_reduced_dim)
		text_reduced = text_reduced.unsqueeze(2)  # (batch_size, 1, 1, text_reduced_dim)
		text_reduced = text_reduced.expand(-1, 4, 4, -1)

		concat_out = torch.cat((d_net_out, text_reduced), 3)  # (1, 4, 4, 512+text_reduced_dim)
		
		logit = self.cat_net(concat_out.permute(0, 3, 1, 2))
		logit = logit.reshape(-1, logit.size()[1] * logit.size()[2] * logit.size()[3])
		logit = self.linear(logit)

		output = torch.sigmoid(logit)

		return output, logit

In [ ]:
!ls

bi_skip.npz	 dictionary.txt		   images1.pickle  sample_data
bi_skip.npz.pkl  drive			   images2.pickle  uni_skip.npz
btable.npy	 flickr30k_images	   images3.pickle  uni_skip.npz.pkl
captions.pickle  flickr-image-dataset.zip  kaggle.json	   utable.npy


In [ ]:
# b = GAN_CLS(50,128,10,20,'./log_dir/','./training_checkpoints','./sample_dir','./final_model',10,300,64, 64,64,0.01,0.9,0.99,a)

# Train.py

In [ ]:
class GAN_CLS(object):
	def __init__(self, args, data_loader, SUPERVISED=True):
		"""
		args : Arguments
		data_loader = An instance of class DataLoader for loading our dataset in batches
		"""

		self.data_loader = data_loader
		self.num_epochs = args.num_epochs
		self.batch_size = args.batch_size

		self.log_step = args.log_step
		self.sample_step = args.sample_step

		self.log_dir = args.log_dir
		self.checkpoint_dir = args.checkpoint_dir
		self.sample_dir = args.sample_dir
		self.final_model = args.final_model
		self.model_save_step = args.model_save_step 

		#self.dataset = args.dataset
		#self.model_name = args.model_name

		self.img_size = args.img_size # 300
		self.z_dim = args.z_dim #64
		self.text_embed_dim = args.text_embed_dim #64
		self.text_reduced_dim = args.text_reduced_dim #64
		self.learning_rate = args.learning_rate #0.01
		self.beta1 = args.beta1 # 0.9
		self.beta2 = args.beta2 # 0.99
		self.l1_coeff = args.l1_coeff 
		self.resume_epoch = args.resume_epoch
		self.resume_idx = args.resume_idx
		self.SUPERVISED = SUPERVISED

		# Logger setting
		log_name = datetime.datetime.now().strftime('%Y-%m-%d')+'.log'
		self.logger = logging.getLogger('__name__')
		self.logger.setLevel(logging.INFO)
		self.formatter = logging.Formatter('%(asctime)s:%(levelname)s:%(message)s')
		self.file_handler = logging.FileHandler(os.path.join(self.log_dir, log_name))
		self.file_handler.setFormatter(self.formatter)
		self.logger.addHandler(self.file_handler)

		self.build_model()

	def smooth_label(self, tensor, offset):
		return tensor + offset
		
	def dump_imgs(images_Array, name):
		with open('{}.pickle'.format(name), 'wb') as file:
			dump(images_Array, file)
		
	def build_model(self):
		""" A function of defining following instances :
		-----  Generator
		-----  Discriminator
		-----  Optimizer for Generator
		-----  Optimizer for Discriminator
		-----  Defining Loss functions
		"""

		# ---------------------------------------------------------------------#
		#						1. Network Initialization					   #
		# ---------------------------------------------------------------------#
		self.gen = Generator(batch_size=self.batch_size,
							 img_size=self.img_size,
							 z_dim=self.z_dim,
							 text_embed_dim=self.text_embed_dim,
							 text_reduced_dim=self.text_reduced_dim)

		self.disc = Discriminator(batch_size=self.batch_size,
								  img_size=self.img_size,
								  text_embed_dim=self.text_embed_dim,
								  text_reduced_dim=self.text_reduced_dim)

		self.gen_optim = optim.Adam(self.gen.parameters(),
									lr=self.learning_rate,
									betas=(self.beta1, self.beta2))

		self.disc_optim = optim.Adam(self.disc.parameters(),
									 lr=self.learning_rate,
									 betas=(self.beta1, self.beta2))

		self.cls_gan_optim = optim.Adam(itertools.chain(self.gen.parameters(),
														self.disc.parameters()),
										lr=self.learning_rate,
										betas=(self.beta1, self.beta2))

		print ('-------------  Generator Model Info  ---------------')
		self.print_network(self.gen, 'G')
		print ('------------------------------------------------')

		print ('-------------  Discriminator Model Info  ---------------')
		self.print_network(self.disc, 'D')
		print ('------------------------------------------------')

		self.criterion = nn.BCELoss().cuda()
		# self.CE_loss = nn.CrossEntropyLoss().cuda()
		# self.MSE_loss = nn.MSELoss().cuda()
		self.gen.train()
		self.disc.train()

	def print_network(self, model, name):
		""" A function for printing total number of model parameters """
		num_params = 0
		for p in model.parameters():
			num_params += p.numel()

		print(model)
		print(name)
		print("Total number of parameters: {}".format(num_params))

	def load_checkpoints(self, resume_epoch, idx):
		"""Restore the trained generator and discriminator."""
		print('Loading the trained models from epoch {} and iteration {}...'.format(resume_epoch, idx))
		G_path = os.path.join(self.checkpoint_dir, '{}-{}-G.ckpt'.format(resume_epoch, idx))
		D_path = os.path.join(self.checkpoint_dir, '{}-{}-D.ckpt'.format(resume_epoch, idx))
		self.gen.load_state_dict(torch.load(G_path, map_location=lambda storage, loc: storage))
		self.disc.load_state_dict(torch.load(D_path, map_location=lambda storage, loc: storage))

	def train_model(self):

		data_loader = self.data_loader

		start_epoch = 0
		if self.resume_epoch >= 0:
			start_epoch = self.resume_epoch
			self.load_checkpoints(self.resume_epoch, self.resume_idx)

		print ('---------------  Model Training Started  ---------------')
		start_time = time.time()

		for epoch in range(start_epoch, self.num_epochs):
			print("Epoch: {}".format(epoch+1))
			for idx, batch in enumerate(data_loader):
				print("Index: {}".format(idx+1), end = "\t")
				true_imgs = batch['true_imgs']
				true_embed = batch['true_embds']
				false_imgs = batch['false_imgs']

				real_labels = torch.ones(true_imgs.size(0))
				fake_labels = torch.zeros(true_imgs.size(0))

				smooth_real_labels = torch.FloatTensor(self.smooth_label(real_labels.numpy(), -0.1))

				true_imgs = Variable(true_imgs.float()).cuda()
				true_embed = Variable(true_embed.float()).cuda()
				false_imgs = Variable(false_imgs.float()).cuda()

				real_labels = Variable(real_labels).cuda()
				smooth_real_labels = Variable(smooth_real_labels).cuda()
				fake_labels = Variable(fake_labels).cuda()

				# ---------------------------------------------------------------#
				# 					  2. Training the generator                  #
				# ---------------------------------------------------------------#
				self.gen.zero_grad()
				z = Variable(torch.randn(true_imgs.size(0), self.z_dim)).cuda()
				fake_imgs = self.gen.forward(true_embed, z)
				fake_out, fake_logit = self.disc.forward(fake_imgs, true_embed)
				fake_out = Variable(fake_out.data, requires_grad=True).cuda()
				
				true_out, true_logit = self.disc.forward(true_imgs, true_embed)
				true_out = Variable(true_out.data, requires_grad=True).cuda()
				
				g_sf = self.criterion(fake_out, real_labels)
				#g_img = self.l1_coeff * nn.L1Loss()(fake_imgs, true_imgs)
				gen_loss = g_sf

				gen_loss.backward()
				self.gen_optim.step()

				# ---------------------------------------------------------------#
				# 					3. Training the discriminator				 #
				# ---------------------------------------------------------------#
				self.disc.zero_grad()
				false_out, false_logit = self.disc.forward(false_imgs, true_embed)
				false_out = Variable(false_out.data, requires_grad=True)
				
				sr = self.criterion(true_out, smooth_real_labels)
				sw = self.criterion(true_out, fake_labels)
				sf = self.criterion(false_out, smooth_real_labels)
				
				disc_loss =  torch.log(sr) + (torch.log(1-sw) + torch.log(1-sf ))/2 

				disc_loss.backward()
				self.disc_optim.step()

				self.cls_gan_optim.step()

				# Logging
				loss = {}
				loss['G_loss'] = gen_loss.item()
				loss['D_loss'] = disc_loss.item()

				# ---------------------------------------------------------------#
				# 					4. Logging INFO into log_dir				 #
				# ---------------------------------------------------------------#
				log = ""
				if (idx + 1) % self.log_step == 0:
					end_time = time.time() - start_time
					end_time = datetime.timedelta(seconds=end_time)
					log = "Elapsed [{}], Epoch [{}/{}], Idx [{}]".format(end_time, epoch + 1, self.num_epochs, idx)

				for net, loss_value in loss.items():
					log += "{}: {:.4f}".format(net, loss_value)
					self.logger.info(log)
					print (log)
				
				
				# ---------------------------------------------------------------#
				# 					5. Saving generated images					 #
				# ---------------------------------------------------------------#
				if (idx + 1) % self.sample_step == 0:
					concat_imgs = torch.cat((true_imgs, fake_imgs), 0)  # ??????????
					concat_imgs = (concat_imgs + 1) / 2
					# out.clamp_(0, 1)
					 
					save_path = os.path.join(self.sample_dir, '{}-{}-images.jpg'.format(epoch, idx + 1))
					# concat_imgs.cpu().detach().numpy()
					self.dump_imgs(concat_imgs.cpu().numpy(), save_path)
					
					#save_image(concat_imgs.data.cpu(), self.sample_dir, nrow=1, padding=0)
					print ('Saved real and fake images into {}...'.format(self.sample_dir))
				
				
				# ---------------------------------------------------------------#
				# 				6. Saving the checkpoints & final model			 #
				# ---------------------------------------------------------------#
				if (idx + 1) % self.model_save_step == 0:
					G_path = os.path.join(self.checkpoint_dir, '{}-{}-G.ckpt'.format(epoch, idx + 1))
					D_path = os.path.join(self.checkpoint_dir, '{}-{}-D.ckpt'.format(epoch, idx + 1))
					torch.save(self.gen.state_dict(), G_path)
					torch.save(self.disc.state_dict(), D_path)
					print('Saved model checkpoints into {}...\n'.format(self.checkpoint_dir))

		print ('---------------  Model Training Completed  ---------------')
		# Saving final model into final_model directory
		G_path = os.path.join(self.final_model, '{}-G.pth'.format('final'))
		D_path = os.path.join(self.final_model, '{}-D.pth'.format('final'))
		torch.save(self.gen.state_dict(), G_path)
		torch.save(self.disc.state_dict(), D_path)
		print('Saved final model into {}...'.format(self.final_model))
  


# Main.py

In [ ]:
from torch.utils.data import DataLoader

#-----------------------------------------------------------------
def check_dir(dir_name):
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)

    print ('{} created'.format(dir_name))

#-----------------------------------------------------------------
def check_args(args):
    # Make all directories if they don't exist

    # --checkpoint_dir
    check_dir(args.checkpoint_dir)

    # --sample_dir
    check_dir(args.sample_dir)

    # --log_dir
    check_dir(args.log_dir)

    # --final_model dir
    check_dir(args.final_model)

    # --epoch
    assert args.num_epochs > 0, 'Number of epochs must be greater than 0'

    # --batch_size
    assert args.batch_size > 0, 'Batch size must be greater than zero'

    # --z_dim
    assert args.z_dim > 0, 'Size of the noise vector must be greater than zero'

    return args


In [ ]:
import argparse
import sys
def main():
    parser = argparse.ArgumentParser()
    # parser.add_argument_group('Dataset related arguments')
    # parser.add_argument('--data_dir', type=str, default="Data",
    #                     help='Data Directory')

    # parser.add_argument('--dataset', type=str, default="flowers",
    #                     help='Dataset to train')

    parser.add_argument_group('Model saving path and steps related arguments')
    parser.add_argument('--log_step', type=int, default=100,
                        help='Save INFO into logger after every x iterations')

    parser.add_argument('--sample_step', type=int, default=100,
                        help='Save generated image after every x iterations')

    parser.add_argument('--checkpoint_dir', type=str, default='checkpoints',
                        help='Save model checkpoints after every x iterations')

    parser.add_argument('--sample_dir', type=str, default='sample',
                        help='Save generated image after every x iterations')

    parser.add_argument('--log_dir', type=str, default='logs',
                        help='Save INFO into logger after every x iterations')

    parser.add_argument('--final_model', type=str, default='final_model',
                        help='Save INFO into logger after every x iterations')

    parser.add_argument_group('Model training related arguments')
    parser.add_argument('--num_epochs', type=int, default=200,
                        help='Total number of epochs to train')

    parser.add_argument('--batch_size', type=int, default=1,
                        help='Batch Size')

    parser.add_argument('--img_size', type=int, default=64,
                        help='Size of the image')

    parser.add_argument('--z_dim', type=int, default=100,
                        help='Size of the latent variable')

    parser.add_argument('--text_embed_dim', type=int, default=4800,
                        help='Size of the embeddding for the captions')

    parser.add_argument('--text_reduced_dim', type=int, default=1024,
                        help='Reduced dimension of the caption encoding')

    parser.add_argument('--learning_rate', type=float, default=0.0002,
                        help='Learning Rate')

    parser.add_argument('--beta1', type=float, default=0.5,
                        help='Hyperparameter of the Adam optimizer')

    parser.add_argument('--beta2', type=float, default=0.999,
                        help='Hyperparameter of the Adam optimizer')

    parser.add_argument('--l1_coeff', type=float, default=50,
                        help='Coefficient for the L1 Loss')

    parser.add_argument('--resume_epoch', type=int, default=1,
                        help='Resume epoch to resume training')

    args = parser.parse_args()

    check_args(args)

    dataset = Text2ImageDataset()
    data_loader = DataLoader(dataset, batch_size=64, shuffle=True)

    gan = GAN_CLS(args, data_loader)

    gan.build_model()
    gan.train_model()



In [ ]:
if __name__ == '__main__':
    main()

usage: ipykernel_launcher.py [-h] [--log_step LOG_STEP]
                             [--sample_step SAMPLE_STEP]
                             [--checkpoint_dir CHECKPOINT_DIR]
                             [--sample_dir SAMPLE_DIR] [--log_dir LOG_DIR]
                             [--final_model FINAL_MODEL]
                             [--num_epochs NUM_EPOCHS]
                             [--batch_size BATCH_SIZE] [--img_size IMG_SIZE]
                             [--z_dim Z_DIM] [--text_embed_dim TEXT_EMBED_DIM]
                             [--text_reduced_dim TEXT_REDUCED_DIM]
                             [--learning_rate LEARNING_RATE] [--beta1 BETA1]
                             [--beta2 BETA2] [--l1_coeff L1_COEFF]
                             [--resume_epoch RESUME_EPOCH]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-5f7f71cd-01f2-4362-9a0f-e878e21d66eb.json


SystemExit: ignored

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
dataset = Text2ImageDataset()
data_loader = DataLoader(dataset, batch_size=64, shuffle=True)


In [ ]:
# Instead of using the args variables, use the values to be passed into the class of train